TO DO:

1. Create new combined hdf5 file with TRAIN/TEST groups with attributes attached to each galaxy group (save in ceph, not home)
2. Create pytorch dataset class to index the hdf file and return (x, y) samples
3. Create pytorch dataloaders for train/val/test in <code> /mnt/home/dmohan/dreams-dm/dreamsdm/datamodules.py </code>





In [1]:
import h5py
from pathlib import Path
import pandas as pd
import os
import numpy as

/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <

In [2]:
## get all the file paths
# file_path = '/mnt/ceph/users/fvillaescusa/DREAMS/Software/zoom_images/'
# recursive = True
# p = Path(file_path) #path to folder containing files
# assert(p.is_dir())
# if(recursive):
#     files = sorted(p.glob('**/*.hdf5'))
# else:
#     files = sorted(p.glob('*.hdf5'))
# if len(files)<1:
#     raise RuntimeError('No hdf5 data found')
# print(files)

create new hdf5 file

pseudo code for writing hdf files 

creating groups for train/test

```python
with h5py.File(new_file_path, 'w') as hdf:
    train_group = hdf.create_group('Train')train_group = hdf.create_group('Train')train_group = hdf.create_group('Train')
    train_group.create_dataset('for Galaxy0 to Galaxy800', data = 'for Mgas0 to Mgas342' )
    #set attributes

    test_group = hdf.create_group('Test')
    test_group.create_dataset('for Galaxy 800 to 1024', data = 'for Mgas0 to Mgas342')

    #all datasets (Mgas0 to Mgas342) have the same arrtibutes for a particular simulation
    dataset.attrs['WDM'] = wdm_mass 
    dataset.attrs['AGN'] = agn
    dataset.attrs['SN1/SN2'] = sn1
```

In [3]:
file_path = '/mnt/ceph/users/fvillaescusa/DREAMS/Software/zoom_images/'
new_file_path = '/mnt/ceph/users/dmohan/dreams/data/dreams/mwzooms_test.hdf5'
df = pd.read_csv('/mnt/home/dmohan/dreams-dm/dreamsdm/data/wdm/labels.csv')
labels = df

In [4]:
labels.iloc[0,0]

'Galaxy_0'

In [5]:
with h5py.File(new_file_path, "w") as hdf:
    
    train_group = hdf.create_group('Train')
    for idx in range(0, 800): #0 to 800
        simid = labels.iloc[idx,0]
        galaxy_path = os.path.join(file_path, simid)
        galaxy_file = h5py.File(galaxy_path+'.hdf5', 'r')    
        sim_group = train_group.create_group(simid)
        # sim_group.attrs["WDM"] = labels.iloc[idx,1]
        # sim_group.attrs["SN1"] = labels.iloc[idx,2]
        # sim_group.attrs["SN2"] = labels.iloc[idx,3]
        # sim_group.attrs["AGN"] = labels.iloc[idx,4]

        for i in range(0, 2): #0 to 343
            mgas = 'Mgas_'+str(i)
            image = np.array(galaxy_file[mgas])
            dataset = sim_group.create_dataset(mgas, data = image )
            dataset.attrs["WDM"] = labels.iloc[idx,1]
            dataset.attrs["SN1"] = labels.iloc[idx,2]
            dataset.attrs["SN2"] = labels.iloc[idx,3]
            dataset.attrs["AGN"] = labels.iloc[idx,4]

    test_group = hdf.create_group('Test')

    for idx in range(800,1024): #800 to 1024
        if(idx not in [864, 874, 884, 894, 904, 914, 924, 934, 944, 954, 964, 974, 984, 994, 1004, 1014]):
            simid = labels.iloc[idx,0]
            galaxy_path = os.path.join(file_path, simid)
            galaxy_file = h5py.File(galaxy_path+'.hdf5', 'r')   
            sim_group = test_group.create_group(simid)
            
            for i in range(0, 2):
                mgas = 'Mgas_'+str(i)
                image = np.array(galaxy_file[mgas])
                dataset = sim_group.create_dataset(mgas, data = image )
                dataset.attrs["WDM"] = labels.iloc[idx,1]
                dataset.attrs["SN1"] = labels.iloc[idx,2]
                dataset.attrs["SN2"] = labels.iloc[idx,3]
                dataset.attrs["AGN"] = labels.iloc[idx,4]
hdf.close()


In [6]:
with h5py.File(new_file_path, "r") as hdf:
    ls = list(hdf.keys())
    print(ls)
    
    print(list(hdf['Train'].keys()))
    print(list(hdf['Train']['Galaxy_0'].keys()))
    data = hdf['Train']['Galaxy_0']['Mgas_0'] #.get('Mgas_0')
    print(data)
    # print(np.array(data))
    print(hdf['Train']['Galaxy_0']['Mgas_0'] .attrs["WDM"])

    print(list(hdf['Test'].keys()))
    print(list(hdf['Test']['Galaxy_800'].keys()))
    data = hdf['Test']['Galaxy_800'].get('Mgas_0')
    print(data)
    print(hdf['Test']['Galaxy_800']['Mgas_0'].attrs["WDM"])

['Test', 'Train']
['Galaxy_0', 'Galaxy_1', 'Galaxy_10', 'Galaxy_100', 'Galaxy_101', 'Galaxy_102', 'Galaxy_103', 'Galaxy_104', 'Galaxy_105', 'Galaxy_106', 'Galaxy_107', 'Galaxy_108', 'Galaxy_109', 'Galaxy_11', 'Galaxy_110', 'Galaxy_111', 'Galaxy_112', 'Galaxy_113', 'Galaxy_114', 'Galaxy_115', 'Galaxy_116', 'Galaxy_117', 'Galaxy_118', 'Galaxy_119', 'Galaxy_12', 'Galaxy_120', 'Galaxy_121', 'Galaxy_122', 'Galaxy_123', 'Galaxy_124', 'Galaxy_125', 'Galaxy_126', 'Galaxy_127', 'Galaxy_128', 'Galaxy_129', 'Galaxy_13', 'Galaxy_130', 'Galaxy_131', 'Galaxy_132', 'Galaxy_133', 'Galaxy_134', 'Galaxy_135', 'Galaxy_136', 'Galaxy_137', 'Galaxy_138', 'Galaxy_139', 'Galaxy_14', 'Galaxy_140', 'Galaxy_141', 'Galaxy_142', 'Galaxy_143', 'Galaxy_144', 'Galaxy_145', 'Galaxy_146', 'Galaxy_147', 'Galaxy_148', 'Galaxy_149', 'Galaxy_15', 'Galaxy_150', 'Galaxy_151', 'Galaxy_152', 'Galaxy_153', 'Galaxy_154', 'Galaxy_155', 'Galaxy_156', 'Galaxy_157', 'Galaxy_158', 'Galaxy_159', 'Galaxy_16', 'Galaxy_160', 'Galaxy_161'

In [12]:
index = 

if(index not in [864, 874]):
    print(index)

875
